# 程序 `train_multinet`

这个程序是实际执行训练的程序。它也包含了对数据的最后处理。

之所以这个程序还会对数据作最后处理，是因为在程序实践中，将所有数据储存成为一个大的 .pt 文件是一件非常消耗硬盘 I/O 的事情。有一段时间服务器的 I/O 很成问题，因此我决定先把数据作粗处理，但粗处理的信息应当是完整的。最后的微调就留待训练过程中进行了；以后要重新调整模型也不需要重新生成 .pt 文件了。

由于训练时，微调是在 CPU 进行而训练优参过程在 GPU 进行，因此一般来说两者之间互不影响；微调对程序效率的影响 (CPU) 效率一般认为比 GPU 效率来得不重要，因此其实是几乎不影响效率的。

微调包含

- 由于数据集的数据可能较为庞大 (譬如训练数据中出现 20 等比较大的数值，但一般的训练数据希望在 -1 到 1 之间) 而重新缩放

- 需要对一些距离作修改 (譬如作色散力修改) 或展开 (将 Gaussian 基组的函数值在一些距离处展开)

In [1]:
import numpy as np
import os

import torch
from alchemy_data import *
from train_multinet import *

RDKit WARNING: [20:55:14] Enabling RDKit 2019.09.1 jupyter extensions


## 函数对象 `AtomTransform`

这是对原子构成图的数据的微调程序。

In [2]:
dev_atom_dataset = AlchemyData(mode='dev', net_type="atom", train_csv_path="./raw/train.csv")
dev_atom_dataset[963]

Data(edge_attr=[289, 7], edge_index=[2, 289], x=[17, 13], y=[1, 12])

In [3]:
AtomTransform()(dev_atom_dataset[963])

Data(edge_attr=[289, 10], edge_index=[2, 289], x=[17, 13], y=[1, 12])

函数对象不对 `data.x` 做处理，但对 `data.edge_attr` 作扩维。其修改有

- 0：$r_{AB} / 25$

- 1:7：保留原数值

- 7：$5 / r_{AB}$

- 8：$50 e^{-r_{AB}}$

- 9：$10 / r_{AB}^6$

## 函数对象 `OrbitalTransform`

这是对轨道构成图的数据的微调程序。

In [4]:
dev_orbital_dataset = AlchemyData(mode='dev', net_type="orbital", train_csv_path="./raw/train.csv")
dev_orbital_dataset[963]

Data(atom_idx=[65], edge_attr=[4225, 8], edge_index=[2, 4225], x=[65, 13], y=[1, 12])

In [5]:
OrbitalTransform()(dev_orbital_dataset[963])

Data(atom_idx=[65], edge_attr=[4225, 10], edge_index=[2, 4225], x=[65, 21], y=[1, 12])

函数对象对 `data.x` 扩 8 维度，扩充的维度数据是 $s_n e^{- \zeta m_n}, \, (n = 0, 1, \cdots, 7)$。其中，$s_n$ 是缩放数据，$m_n$ 为指数数据：

```python
[10, 15, 25, 50, 100, 250, 2500, 100000]  # s_n
[1, 1.5, 2, 2.5, 3, 4, 6, 9]              # m_n
```

对 `data.edge_attr` 扩 2 维度，扩充维度数据是 $5 / r_{AB}$ 与 $50 e^{-r_{AB}}$。

同时，对动能积分、偶极积分、以及轨道中心间距离均除以 25，以防止过大的积分值。

## 迭代器 `MyRandomSampler`

我们的模型涉及到两个耦合的图：原子构成的图与轨道构成的图。同时，我们希望在训练时打乱数据，以保证训练的质量。但我们不能随意地打乱数据，因为不能让原子图与轨道图所对应的分子是不同的。因此，原子图数据集与轨道图数据集的乱序必须是相同的乱序。

相同的乱序可以依据相同的随机种子所给出。这个迭代器事实上就是，在一次 epoch 时，先给定一个种子，让原子图与轨道图都使用这个相同的种子，然后根据这个种子进行迭代训练。待一次 epoch 结束后，下一次 epoch 时，就重新定义这个种子，以对数据集作重新排序。

## 关于训练程序的一些说明

### 关于 5-fold Cross-Validation 的说明

这个程序使用 5-fold CV 的做法。具体来说，其数据集被分为 `dev`, `valid_00`, `valid_01`, `valid_02`, `valid_03`, `valid_04`, `test` 七个。`dev` 和 `test` 就是 Alchemy 竞赛的 `dev` 和 `test` 数据集 (`test` 数据集没有 target，即不知道预期结果)，而 `valid_0*` 则是 Alchemy 竞赛 `valid` 数据集经过 (伪) 随机的划分而成，互不相交且并集为 `valid` 数据集。

若我们进行 02 号验证集学习，我们会将 `dev`, `valid_00`, `valid_01`, `valid_03`, `valid_04` 五个数据集合并为一个完整的训练集，而 `valid_02` 则为验证集，`test` 为测试集。

对于其它验证集的情况也相同。最后汇报的测试集结果是由这五个验证集 5-fold 所得到数据的平均值。

程序中，Line 188 - Line 217 就在说明这个问题。

对于原子构成的图的数据，我们首先将数据集储存到 `dev_atom_dataset`, `valid_atom_dataset_list`, `test_atom_dataset` 中；其中 `valid_atom_dataset_list` 是一个长度为 5 的列表，储存了五个 valid 数据集。

随后，根据命令行中 `--valid` 所输入的数据 (表示选用 0, 1, 2, 3, 4 中何者作为验证集)，将这个数据集划分出来成为验证集 `valid_atom_dataset`，剩下的与 `dev_atom_dataset` 并入训练集 `train_atom_datasets`。

最后，依据 PyTorch 的实现方式，将数据集代入 `DataLoader` 框架，使数据集成为可迭代对象。对于训练集迭代对象 `train_atom_loader`，需要额外设置 `train_atom_sampler` 以使不同的 epoch 的训练过程采用不同的数据排序。

对于轨道构成的图的数据也是类似的处理。

### 训练过程与参数设置

| 参数含义 | 程序所在位置 | 示例程序参数值 | 提交比赛参数值 |
|:----:|:----:|:----:|:----:|
| 原子节点隐含层向量长度 | Line 221 | 18 | 64 |
| 原子边隐含层向量长度   | Line 222 | 12 | 48 |
| 轨道节点隐含层向量长度 | Line 223 | 12 | 48 |
| 轨道边隐含层向量长度   | Line 224 |  8 | 24 |
| MPNN 迭代次数          | Line 226 |  6 |  6 |
| Set2Set 迭代次数       | Line 227 |  6 |  6 |
| Adam 学习率            | Line 230 | 0.0005 | 0.0005 |
| Batch 大小             | Line 172 | 32 | 16 |
| Early-Stopping 大小    | Line 170 |  5 | 25 |
| 最大 Epoch             | Line 171 | 15 | 5000 |

上述参数中，除了最大 Epoch 在足够大时不会不影响训练过程外，其它都会影响训练过程。

对于 Batch 与 Early-Stopping 需要作额外说明。

Batch 在实际执行时不能设得太大。这是为了避免对 GPU 的显存占用太大。这也与隐含层向量长度有关。因此，提交到服务器的 Batch 大小反而只有 16。

一般来说，没有 Batch 越大训练效果越好的说法。较小的 Batch 的优势是在训练初期加快参数优化到比较靠近局域极小点，甚至可能一个 epoch 就能近乎于让模型的 Loss 降得很小；但缺点是训练后期时，模型的参数会很严重地向最后训练的数据过拟合。

Early-Stopping 大小为 25 指的是，如果训练过程中，验证集的 Loss 在连续 25 个 Epoch 下都没有变得更小，那么就终止训练。这无关乎训练集的 Loss 是否仍然变小或变大。

## 实际用于竞赛的程序的一些错误

### `OrbitalTransform` 设置错误

在我们现在的示例程序 train_multinet.py 中，程序的 Line 137 为

```python
orbital_vertex[:, idx + 13] = torch.exp(- zeta * multiplier) * scaler
```

但实际用于竞赛的程序是错误的，为

```python
orbital_vertex[:, idx + 13] = torch.exp(- zeta * multiplier).max() * scaler
```

这可能比较影响训练的效果。

### mpnn_multinet.py 中 MPNN 迭代次数的设置错误

在我们的示例程序 mpnn_multinet.py 中，程序的 Line 85 为

```python
for i in range(self.mp_step):
```

但实际用于竞赛的程序是错误的，为

```python
for i in range(self.s2s_step):
```

由于恰好 MPNN 迭代次数与 Set2Set 迭代次数设置相同，因此对训练过程没有影响。